In [4]:
import pandas as pd
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier, StackingClassifier
from xgboost import XGBClassifier
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
from imblearn.over_sampling import SMOTE
import seaborn as sns
import matplotlib.pyplot as plt
import pre_data as eda

data = pd.read_csv('./data/train.csv', encoding='cp949')
preprocessed_data = eda.preprocessing(data)  # 기본 전처리 수행

# 범주형 데이터를 정수형으로 변환
object_columns = preprocessed_data.select_dtypes(include=['object'])
convert_data, _ = eda.convert_category_into_integer(preprocessed_data, object_columns)

# 1. 필요한 컬럼만 사용 (타겟 변수를 제외하고 X, y 설정)
X = convert_data.drop(columns='Churn')
y = convert_data['Churn'].astype(int)

# 데이터 스케일링
scaler = StandardScaler()
X_scaled = pd.DataFrame(scaler.fit_transform(X), columns=X.columns)

# SMOTE로 클래스 불균형 해결
smote = SMOTE(random_state=0)
X_resampled, y_resampled = smote.fit_resample(X_scaled, y)

# 데이터를 학습용과 테스트용으로 분할
X_train, X_test, y_train, y_test = train_test_split(X_resampled, y_resampled, test_size=0.2, random_state=0)

# 개별 모델 선언
xgb = XGBClassifier(use_label_encoder=False, eval_metric='logloss', random_state=0)
rf = RandomForestClassifier(n_estimators=100, random_state=0)
lr = LogisticRegression(solver='liblinear', max_iter=1000)

# 스태킹 모델 구성 (기본 모델들은 XGBoost와 RandomForest, 최종 메타 모델은 LogisticRegression 사용)
estimators = [
    ('xgb', xgb),
    ('rf', rf)
]

# StackingClassifier 선언 (기본 모델들을 앙상블하고, 최종 모델로 로지스틱 회귀 사용)
stacking_clf = StackingClassifier(
    estimators=estimators,
    final_estimator=lr,
    cv=5,  # 교차 검증 적용
    n_jobs=-1
)

# 스태킹 모델 학습
stacking_clf.fit(X_train, y_train)

# 예측
y_pred = stacking_clf.predict(X_test)

# 성능 평가
accuracy = accuracy_score(y_test, y_pred)
precision = precision_score(y_test, y_pred)
recall = recall_score(y_test, y_pred)
f1 = f1_score(y_test, y_pred)

# 성능 출력
print("\nFinal Model Performance with Stacking:")
print(f"Accuracy: {accuracy}")
print(f"Precision: {precision}")
print(f"Recall: {recall}")
print(f"F1 Score: {f1}")


nan-value filled
MonthlyRevenue : 50887, 삭제된 수 : 1
MonthlyMinutes : 50883, 삭제된 수 : 4
TotalRecurringCharge : 50874, 삭제된 수 : 9
DirectorAssistedCalls : 50874, 삭제된 수 : 0
OverageMinutes : 50870, 삭제된 수 : 4
RoamingCalls : 50862, 삭제된 수 : 8
PercChangeMinutes : 50860, 삭제된 수 : 2
DroppedCalls : 50856, 삭제된 수 : 4
BlockedCalls : 50852, 삭제된 수 : 4
UnansweredCalls : 50842, 삭제된 수 : 10
CustomerCareCalls : 50840, 삭제된 수 : 2
ThreewayCalls : 50836, 삭제된 수 : 4
ReceivedCalls : 50830, 삭제된 수 : 6
OutboundCalls : 50827, 삭제된 수 : 3
InboundCalls : 50823, 삭제된 수 : 4
PeakCallsInOut : 50821, 삭제된 수 : 2
OffPeakCallsInOut : 50806, 삭제된 수 : 15
DroppedBlockedCalls : 50792, 삭제된 수 : 14
CallForwardingCalls : 50779, 삭제된 수 : 13
CallWaitingCalls : 50779, 삭제된 수 : 0
MonthsInService : 50778, 삭제된 수 : 1
UniqueSubs : 50777, 삭제된 수 : 1
ActiveSubs : 50777, 삭제된 수 : 0
Handsets : 50773, 삭제된 수 : 4
HandsetModels : 50773, 삭제된 수 : 0
CurrentEquipmentDays : 50768, 삭제된 수 : 5
RetentionCalls : 50634, 삭제된 수 : 134
RetentionOffersAccepted : 50634, 삭제된 수 : 0


In [10]:
import pandas as pd
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier, StackingClassifier
from xgboost import XGBClassifier
from sklearn.model_selection import train_test_split, GridSearchCV, StratifiedKFold
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, classification_report, confusion_matrix
from imblearn.over_sampling import SMOTE
from imblearn.pipeline import Pipeline  # imblearn의 Pipeline 사용
from sklearn.compose import ColumnTransformer
import pre_data as eda

# 데이터 로드 및 기본 전처리
data = pd.read_csv('./data/train.csv', encoding='cp949')
preprocessed_data = eda.preprocessing(data)

# 특성과 타겟 변수 분리
X = preprocessed_data.drop(columns='Churn')

# 타겟 변수 변환 ('Yes'를 1로, 'No'를 0으로 매핑)
y = preprocessed_data['Churn'].map({'Yes': 1, 'No': 0})

# 수치형 및 범주형 열 구분
numeric_features = X.select_dtypes(include=['int64', 'float64']).columns
categorical_features = X.select_dtypes(include=['object']).columns

# 파이프라인 구축
numeric_transformer = Pipeline(steps=[
    ('scaler', StandardScaler())
])

categorical_transformer = Pipeline(steps=[
    ('onehot', OneHotEncoder(handle_unknown='ignore'))
])

preprocessor = ColumnTransformer(
    transformers=[
        ('num', numeric_transformer, numeric_features),
        ('cat', categorical_transformer, categorical_features)
    ])

# 모델 정의
xgb = XGBClassifier(use_label_encoder=False, eval_metric='logloss', random_state=0)
rf = RandomForestClassifier(random_state=0)
lr = LogisticRegression(solver='liblinear', max_iter=1000)

# 스태킹 분류기 구성
estimators = [
    ('xgb', xgb),
    ('rf', rf)
]

stacking_clf = StackingClassifier(
    estimators=estimators,
    final_estimator=lr,
    cv=5,
    n_jobs=-1
)

# 전체 파이프라인 (imblearn의 Pipeline 사용)
pipeline = Pipeline(steps=[
    ('preprocessor', preprocessor),
    ('smote', SMOTE(random_state=0)),
    ('classifier', stacking_clf)
])  

# 하이퍼파라미터 그리드 설정
param_grid = {
    'classifier__xgb__n_estimators': [50, 100],
    'classifier__xgb__max_depth': [3, 5],
    'classifier__rf__n_estimators': [100, 200],
    'classifier__rf__max_depth': [None, 10],
    'classifier__final_estimator__C': [0.1, 1.0, 10]
}

# Stratified K-Fold 사용
cv = StratifiedKFold(n_splits=5, shuffle=True, random_state=0)

# 그리드 서치
grid_search = GridSearchCV(
    estimator=pipeline,
    param_grid=param_grid,
    cv=cv,
    scㅁoring='f1',
    n_jobs=-1               
)

# 모델 학습
grid_search.fit(X, y)

# 베스트 모델 추출
best_model = grid_search.best_estimator_

# 테스트 데이터 분할
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=0, stratify=y
)

# 베스트 모델로 예측
best_model.fit(X_train, y_train)
y_pred = best_model.predict(X_test)

# 성능 평가
accuracy = accuracy_score(y_test, y_pred)
precision = precision_score(y_test, y_pred)
recall = recall_score(y_test, y_pred)
f1 = f1_score(y_test, y_pred)

print("\n스태킹 모델의 최종 성능:")
print(f"Accuracy: {accuracy}")
print(f"Precision: {precision}")
print(f"Recall: {recall}")
print(f"F1 Score: {f1}")

# 분류 보고서 및 혼동 행렬 출력
print("\n분류 보고서:")
print(classification_report(y_test, y_pred))

print("혼동 행렬:")
print(confusion_matrix(y_test, y_pred))


nan-value filled
MonthlyRevenue : 50887, 삭제된 수 : 1
MonthlyMinutes : 50883, 삭제된 수 : 4
TotalRecurringCharge : 50874, 삭제된 수 : 9
DirectorAssistedCalls : 50874, 삭제된 수 : 0
OverageMinutes : 50870, 삭제된 수 : 4
RoamingCalls : 50862, 삭제된 수 : 8
PercChangeMinutes : 50860, 삭제된 수 : 2
DroppedCalls : 50856, 삭제된 수 : 4
BlockedCalls : 50852, 삭제된 수 : 4
UnansweredCalls : 50842, 삭제된 수 : 10
CustomerCareCalls : 50840, 삭제된 수 : 2
ThreewayCalls : 50836, 삭제된 수 : 4
ReceivedCalls : 50830, 삭제된 수 : 6
OutboundCalls : 50827, 삭제된 수 : 3
InboundCalls : 50823, 삭제된 수 : 4
PeakCallsInOut : 50821, 삭제된 수 : 2
OffPeakCallsInOut : 50806, 삭제된 수 : 15
DroppedBlockedCalls : 50792, 삭제된 수 : 14
CallForwardingCalls : 50779, 삭제된 수 : 13
CallWaitingCalls : 50779, 삭제된 수 : 0
MonthsInService : 50778, 삭제된 수 : 1
UniqueSubs : 50777, 삭제된 수 : 1
ActiveSubs : 50777, 삭제된 수 : 0
Handsets : 50773, 삭제된 수 : 4
HandsetModels : 50773, 삭제된 수 : 0
CurrentEquipmentDays : 50768, 삭제된 수 : 5
RetentionCalls : 50634, 삭제된 수 : 134
RetentionOffersAccepted : 50634, 삭제된 수 : 0


In [ ]:
import pandas as pd
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier, StackingClassifier
from xgboost import XGBClassifier
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
from imblearn.over_sampling import SMOTE
import seaborn as sns
import matplotlib.pyplot as plt
import pre_data as eda

data = pd.read_csv('./data/train.csv', encoding='cp949')
preprocessed_data = eda.preprocessing(data)  # 기본 전처리 수행

# 범주형 데이터를 정수형으로 변환
object_columns = preprocessed_data.select_dtypes(include=['object'])
convert_data, _ = eda.convert_category_into_integer(preprocessed_data, object_columns)

# 1. 필요한 컬럼만 사용 (타겟 변수를 제외하고 X, y 설정)
X = convert_data.drop(columns='Churn')
y = convert_data['Churn'].astype(int)

# 데이터 스케일링
scaler = StandardScaler()
X_scaled = pd.DataFrame(scaler.fit_transform(X), columns=X.columns)

# SMOTE로 클래스 불균형 해결
smote = SMOTE(random_state=0)
X_resampled, y_resampled = smote.fit_resample(X_scaled, y)

# 데이터를 학습용과 테스트용으로 분할
X_train, X_test, y_train, y_test = train_test_split(X_resampled, y_resampled, test_size=0.2, random_state=0)

# 개별 모델 선언
xgb = XGBClassifier(use_label_encoder=False, eval_metric='logloss', random_state=0)
rf = RandomForestClassifier(n_estimators=100, random_state=0)
lr = LogisticRegression(solver='liblinear', max_iter=1000)

# 스태킹 모델 구성 (기본 모델들은 XGBoost와 RandomForest, 최종 메타 모델은 LogisticRegression 사용)
estimators = [
    ('xgb', xgb),
    ('rf', rf)
]

# StackingClassifier 선언 (기본 모델들을 앙상블하고, 최종 모델로 로지스틱 회귀 사용)
stacking_clf = StackingClassifier(
    estimators=estimators,
    final_estimator=lr,
    cv=5,  # 교차 검증 적용
    n_jobs=-1
)

# 스태킹 모델 학습
stacking_clf.fit(X_train, y_train)

# 예측
y_pred = stacking_clf.predict(X_test)

# 성능 평가
accuracy = accuracy_score(y_test, y_pred)
precision = precision_score(y_test, y_pred)
recall = recall_score(y_test, y_pred)
f1 = f1_score(y_test, y_pred)

# 성능 출력
print("\nFinal Model Performance with Stacking:")
print(f"Accuracy: {accuracy}")
print(f"Precision: {precision}")
print(f"Recall: {recall}")
print(f"F1 Score: {f1}")
